<a href="https://colab.research.google.com/github/Harshaveenakondeti/pyspark/blob/main/Aadhar_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark1 Assignment
Objective: 
Solve the following questions.
1. Create a dataframe with Total Aadhaar's generated for each state
2. Create a dataframe with the Total Aadhaar's generated by each enrollment agency
3. Create dataframe with the top 10 districts with maximum Aadhaar's generated for both Male and females?
4. Create a dataframe with Total Aadhaar generated for the top 10 least states
For which age, most Aadhar cards have declined?

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 64.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=17be5cda72ad5b9f05b17b2694280359f148f7b85b5da6e57991106378274394
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#Initializing PySpark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

#Spark Config
conf = SparkConf().setAppName("sample_aadhar")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('Aadhar').getOrCreate()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
import pyspark.sql.functions as f
df = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/content/UIDAI-ENR-DETAIL-20170308.csv")  
df.show(2)
# load csv and save into df

+--------------+--------------------+-------------+---------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|     Registrar|    Enrolment Agency|        State| District|Sub District|Pin Code|Gender|Age|Aadhaar generated|Enrolment Rejected|Residents providing email|Residents providing mobile number|
+--------------+--------------------+-------------+---------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|Allahabad Bank|A-Onerealtors Pvt...|Uttar Pradesh|Allahabad|        Meja|  212303|     F|  7|                1|                 0|                        0|                                1|
|Allahabad Bank|Asha Security Gua...|Uttar Pradesh|Sonbhadra| Robertsganj|  231213|     M|  8|                1|                 0|                        0|                                0|
+--------------+--------------------+---

In [ ]:
df.columns # for column names

['Registrar',
 'Enrolment Agency',
 'State',
 'District',
 'Sub District',
 'Pin Code',
 'Gender',
 'Age',
 'Aadhaar generated',
 'Enrolment Rejected',
 'Residents providing email',
 'Residents providing mobile number']

In [ ]:
df.printSchema()   # for datatypes

root
 |-- Registrar: string (nullable = true)
 |-- Enrolment Agency: string (nullable = true)
 |-- State: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Sub District: string (nullable = true)
 |-- Pin Code: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Aadhaar generated: integer (nullable = true)
 |-- Enrolment Rejected: integer (nullable = true)
 |-- Residents providing email: integer (nullable = true)
 |-- Residents providing mobile number: integer (nullable = true)



## Data Cleaning

In [ ]:
df=df.withColumnRenamed('Enrolment Agency','Enrolment_Agency').withColumnRenamed('Sub District','Sub_District').withColumnRenamed('Pin Code','Pin_Code').withColumnRenamed('Aadhaar generated','Aadhar_generated')
df=df.withColumnRenamed('Enrolment Rejected', 'Enrolment_Rejected').withColumnRenamed('Residents providing email','Residents_providing_email').withColumnRenamed('Residents providing mobile number','Residents_providing_mobile_number')
df.show()

+--------------------+--------------------+-------------+----------+------------+--------+------+---+----------------+------------------+-------------------------+---------------------------------+
|           Registrar|    Enrolment_Agency|        State|  District|Sub_District|Pin_Code|Gender|Age|Aadhar_generated|Enrolment_Rejected|Residents_providing_email|Residents_providing_mobile_number|
+--------------------+--------------------+-------------+----------+------------+--------+------+---+----------------+------------------+-------------------------+---------------------------------+
|      Allahabad Bank|A-Onerealtors Pvt...|Uttar Pradesh| Allahabad|        Meja|  212303|     F|  7|               1|                 0|                        0|                                1|
|      Allahabad Bank|Asha Security Gua...|Uttar Pradesh| Sonbhadra| Robertsganj|  231213|     M|  8|               1|                 0|                        0|                                0|
|      All

## Data Analysis

# Create a dataframe with Total Aadhaar's generated for each state?

In [ ]:
df1= df.filter(df.Aadhar_generated==1)  # only aadhar generated dataframe, so 1
# Create a dataframe with Total Aadhaar's generated for each state
print('Total Aadhar generated for each state')
df1.groupby('State').sum('Aadhar_generated').show()

Total Aadhar generated for each state
+--------------------+---------------------+
|               State|sum(Aadhar_generated)|
+--------------------+---------------------+
|            Nagaland|                   80|
|           Karnataka|                 7455|
|              Odisha|                 6650|
|              Kerala|                  798|
|          Tamil Nadu|                 3341|
|        Chhattisgarh|                 3278|
|      Andhra Pradesh|                  869|
|      Madhya Pradesh|                13641|
|              Punjab|                 4275|
|             Manipur|                  258|
|                 Goa|                   25|
|             Mizoram|                   26|
|Dadra and Nagar H...|                   45|
|    Himachal Pradesh|                  514|
|          Puducherry|                   66|
|             Haryana|                 2325|
|   Jammu and Kashmir|                  445|
|           Jharkhand|                 4911|
|   Arunachal Pra

Madhya Pradesh is having highest total aadhar genarated with number of 13641.Goa is having least number of total aadhars generated with number of 25.

# Create a dataframe with the Total Aadhaar's generated by each enrollment agency

In [ ]:
# Create a dataframe with the Total Aadhaar's generated by each enrollment agency
print('Total aadhars generated by each enrollment agency')
df1.groupby('Enrolment_Agency').sum('Aadhar_generated').show()

Total aadhars generated by each enrollment agency
+--------------------+---------------------+
|    Enrolment_Agency|sum(Aadhar_generated)|
+--------------------+---------------------+
|    APOnline Limited|                  234|
|  Transmoovers India|                    5|
|     Alankit Limited|                   20|
|Twinstar Industri...|                 2557|
|OSWAL COMPUTERS &...|                  348|
|SHRIKRISHNA KHAND...|                    1|
|Atalji Janasnehi ...|                 1172|
|Sri Ramraja Sarka...|                    1|
|             CSC SPV|                72155|
|Asha Security Gua...|                  586|
|   CMS Computers Ltd|                 6852|
|Bloom Solutions P...|                  404|
|Abha Systems And ...|                   61|
|               BASIX|                 4643|
|Frontech Systems ...|                   67|
|Vedavaag Systems ...|                 3632|
|A I Soc for Elect...|                 3367|
|AVVAS INFOTECH PV...|                 1411|
|    

CSC SPV is enrollment agency having haighest enrollments with 72155.SHRIKRISHNA KHAND & Sri Ramaraja Sarka is having lowest number of enrolments addhar generated.

# Create dataframe with the top 10 districts with maximum Aadhaar's generated for both Male and females?

In [ ]:
# Create dataframe with the top 10 districts with maximum Aadhaar's generated for both Male and females?
df_M= df1.filter(df1.Gender=='M').select('District','Aadhar_generated')
print('filter dataframe for males')
df_M.show(10)
from pyspark.sql.functions import sum, col, desc
print('Top 10 districts for maximum AAdhar generated for Males')
df_M.groupby('District').agg(sum('Aadhar_generated').alias('total_aadhar')).sort(desc('total_aadhar')).show(10)


filter dataframe for males
+----------+----------------+
|  District|Aadhar_generated|
+----------+----------------+
| Sonbhadra|               1|
|    Shamli|               1|
| Gorakhpur|               1|
|  Varanasi|               1|
|  Varanasi|               1|
|  Varanasi|               1|
|  Varanasi|               1|
|Bara Banki|               1|
|  Marigaon|               1|
| Gopalganj|               1|
+----------+----------------+
only showing top 10 rows

Top 10 districts for maximum AAdhar generated for Males
+-----------------+------------+
|         District|total_aadhar|
+-----------------+------------+
|       Barddhaman|        2193|
|North 24 Parganas|        1746|
|            Patna|        1644|
|            Nadia|        1530|
|        Bhagalpur|        1434|
|          Kolkata|        1392|
|       Samastipur|        1380|
|             Gaya|        1322|
|   West Champaran|        1232|
|   East Champaran|        1226|
+-----------------+------------+
only show

For **Males** Barddhaman having highest number of total aadhar generated.

In [ ]:
df_F= df1.filter(df1.Gender=='F').select('District','Aadhar_generated')
print('Filtered dataframe for Females')
df_F.show(10)
from pyspark.sql.functions import sum, col, desc
print('Top Districts for maximum aadhar generated for Females')
df_F.groupby('District').agg(sum('Aadhar_generated').alias('total_aadhar')).sort(desc('total_aadhar')).show(10)

Filtered dataframe for Females
+---------+----------------+
| District|Aadhar_generated|
+---------+----------------+
|Allahabad|               1|
|Sultanpur|               1|
| Bagalkot|               1|
| Bagalkot|               1|
| Bagalkot|               1|
| Bagalkot|               1|
| Bagalkot|               1|
| Bagalkot|               1|
| Bagalkot|               1|
| Bagalkot|               1|
+---------+----------------+
only showing top 10 rows

Top Districts for maximum aadhar generated for Females
+-----------------+------------+
|         District|total_aadhar|
+-----------------+------------+
|North 24 Parganas|        1727|
|       Barddhaman|        1580|
|            Nadia|        1002|
|South 24 Parganas|         958|
|        Bhagalpur|         766|
|           Howrah|         740|
|            Patna|         701|
|          Kolkata|         679|
|Paschim Medinipur|         658|
|          Hooghly|         657|
+-----------------+------------+
only showing top 10 

For **Females** North 24 Parganas having highest total number of aadhars.

#Create a dataframe with Total Aadhaar generated for the top 10 least ages For which age, most Aadhar cards have declined?

In [ ]:
#Create a dataframe with Total Aadhaar generated for the top 10 least states For which age, most Aadhar cards have declined?
new_df = df.where(df.Aadhar_generated==0).select("Age","Aadhar_generated","State")  # 0 for aadhar rejected
print('Filtered dataframe for declined aadhars')
new_df.show(10)
from pyspark.sql.functions import count, asc
print('age with most declined')
new_df.groupBy('Age').agg(count('Aadhar_generated').alias('Aadhar_reject')).sort(desc('Aadhar_reject')).show(10,truncate=False)

Filtered dataframe for declined aadhars
+---+----------------+-------------+
|Age|Aadhar_generated|        State|
+---+----------------+-------------+
| 10|               0|Uttar Pradesh|
| 56|               0|    Karnataka|
| 73|               0|    Karnataka|
| 45|               0|    Karnataka|
| 65|               0|    Karnataka|
|  4|               0|    Karnataka|
| 38|               0|    Karnataka|
| 17|               0|    Karnataka|
| 49|               0|    Karnataka|
|  2|               0|    Karnataka|
+---+----------------+-------------+
only showing top 10 rows

age with most declined
+---+-------------+
|Age|Aadhar_reject|
+---+-------------+
|4  |954          |
|3  |824          |
|2  |744          |
|1  |709          |
|0  |568          |
|5  |496          |
|6  |441          |
|7  |402          |
|8  |326          |
|9  |284          |
+---+-------------+
only showing top 10 rows



At age of 4 , there are most number of rejections with 954.


# Conclusion:
We first initialize Spark Session and spark context and creates dataframe by loading csv file.we read column names and print schemas. we changed few column names having spaces. we filtered daatframe for aadhars generated, which is 1. we created new dataframe for total aadhars for each state using groupby and sum function. we created dataframe for total aadhars for Enrolment agency. We created daatframes for male & female addhars dataset and found maximun total aadhars generated for each district.we filtered data for aadhar rejections. we created dataframe for ages having least age with highest aadhaar rejections.for age 4 we are having highest rejections of 954.


**Author**: Harsha Veena

**IDE**: Google colab

**language**: spark

**Functions used**: filter, where, groupby, desc, sort, sum,agg,alias,withColumnRenamed

**created on**: 28:6:2022